# 2-Data Wrangling Script
Date created: 16/04/23  
Created by: Jayden Dzierbicki  
Last updated: 16/04/23  

The purpose of this notebook is to undertake the following in prepration for machine learning
- Data wrangling and cleaning
- NLP techniques 
- EDA
- Clean data saved and stored into mySQL/csv


In [15]:
import getpass
from sqlalchemy import create_engine # For connecting to SQL databases
import pandas as pd

# Function to load data into mySQL
def extract_data_mySQL(database_name, table_name):
    user = 'root'

    # Prompt the user for a password
    password = getpass.getpass("Enter your MySQL password: ")
    host = 'localhost'
    port = 3306
    database = database_name
    engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

    # Write the DataFrame to a SQL table
    table_name =  table_name
    query = f"SELECT * FROM {table_name}"
    
    # Read query to dataframe
    df = pd.read_sql(query, engine)
    return(df)

    # Close the connection
    engine.dispose()

## Data wrangling and cleaning
- Load in data
- Quick EDA of loaded data
- Decsions based on EDA - period of analysis
- Test and remove duplicates i.e. user could spam a forum board 
- NLP

Decsions made in this section include:
- Analysis from 2021-02-02 -> 2023-04-14: Limitations of data retrival for cryptocompare.com

In [80]:
# Load in data sources
cryptocompare_xrp = extract_data_mySQL('MA5851_A3', 'cryptocompare_xrp')
investingcom_xrp = extract_data_mySQL('MA5851_A3', 'investingcom_xrp')
xrp_price_yahoo = extract_data_mySQL('MA5851_A3', 'xrp_price_yahoo')

Enter your MySQL password: ········
Enter your MySQL password: ········
Enter your MySQL password: ········


In [83]:
def get_min_max_dates(table, table_name):
    print(f'Min and max date range of {table_name}')
    print(f'MIN Date: {table.date.min()}')
    print(f'MAX Date: {table.date.max()}')
    print("-----------------------------")


get_min_max_dates(cryptocompare_xrp, "cryptocompare_xrp")
get_min_max_dates(investingcom_xrp, "investingcom_xrp")
get_min_max_dates(xrp_price_yahoo, "xrp_price_yahoo")


Min and max date range of cryptocompare_xrp
MIN Date: 2021-02-02
MAX Date: 2023-04-14
-----------------------------
Min and max date range of investingcom_xrp
MIN Date: 2018-03-16 00:00:00
MAX Date: 2023-04-14 00:00:00
-----------------------------
Min and max date range of xrp_price_yahoo
MIN Date: 2017-11-10 00:00:00
MAX Date: 2023-04-14 00:00:00
-----------------------------


In [89]:
# Filter all data by date range, using data from 2021-02-02 onwards
min_date_filter = pd.to_datetime('2021-02-02')  # Used to filter min date

def filter_by_min_date(date_filter, df, table_name):
    df['date'] = pd.to_datetime(df['date'])
    filtered_df = df[df['date'] >= date_filter]
    rows_removed = len(df) - len(filtered_df)
    print(f"Number of rows filtered by date from {table_name}: {rows_removed}")
    print("---------------------------------------------------------")
    return filtered_df

filtered_investingcom_xrp = filter_by_min_date(min_date_filter, investingcom_xrp, "investingcom_xrp")
filtered_xrp_price_yahoo = filter_by_min_date(min_date_filter, xrp_price_yahoo, "xrp_price_yahoo")
filtered_cryptocompare_xrp = filter_by_min_date(min_date_filter, cryptocompare_xrp, "cryptocompare_xrp")


Number of rows filtered by date from investingcom_xrp: 26268
---------------------------------------------------------
Number of rows filtered by date from xrp_price_yahoo: 1180
---------------------------------------------------------
Number of rows filtered by date from cryptocompare_xrp: 0
---------------------------------------------------------


In [90]:
# Remove NA observations from comments
def remove_na_values(df, table_name, variable):
    output = df.dropna(subset=[variable])
    rows_removed = len(df) - len(output)
    print(f"Number of NA rows removed from {table_name}: {rows_removed}")
    print("---------------------------------------------------------")
    return output

# Assuming cryptocompare_xrp is a pandas DataFrame with a 'comment' column
cleaned_cryptocompare_xrp = remove_na_values(filtered_investingcom_xrp, "cryptocompare_xrp", "comment")
cleaned_investingcom_xrp = remove_na_values(filtered_cryptocompare_xrp, "investingcom_xrp", "comment")


Number of NA rows removed from cryptocompare_xrp: 43
---------------------------------------------------------
Number of NA rows removed from investingcom_xrp: 129
---------------------------------------------------------


In [108]:
def preview_duplicate_comments(df, table_name):
    # Find duplicates based on 'date' and 'comment' columns
    duplicates = df[df.duplicated(subset=['date', 'comment'], keep=False)]

    # Group the duplicates by 'date' and 'comment' columns and count their occurrences
    duplicate_counts = duplicates.groupby(['date', 'comment']).size().reset_index(name='count')

    # Sort the DataFrame by the 'count' column in descending order
    duplicate_counts_sorted = duplicate_counts.sort_values(by='count', ascending=False)

    # Print the count for each specific duplicate
    print(f"Duplicate preview for {table_name}")
    print(duplicate_counts_sorted)
    print("-------------------------------------------------------------------------")

def remove_duplicate_comments(df, table_name):
    # Remove duplicates
    unique_df = df.drop_duplicates(subset=['date', 'comment'])
    duplicates_removed = len(df) - len(unique_df)
    print(f'Number of duplicates removed from {table_name}: {duplicates_removed}')
    print("-------------------------------------------------------------------------")
    return unique_df

# Preview and remove duplicates
preview_duplicate_comments(cleaned_cryptocompare_xrp, "cleaned_cryptocompare_xrp")
preview_duplicate_comments(cleaned_investingcom_xrp, "cleaned_investingcom_xrp")

cleaned_cryptocompare_xrp_2 = remove_duplicate_comments(cleaned_cryptocompare_xrp, "cleaned_cryptocompare_xrp")
cleaned_investingcom_xrp_2 = remove_duplicate_comments(cleaned_investingcom_xrp, "cleaned_investingcom_xrp")

Duplicate preview for cleaned_cryptocompare_xrp
          date                                            comment  count
124 2021-04-13                                                lol      6
96  2021-04-10                                                lol      5
38  2021-02-21                                            shut up      4
105 2021-04-11  My newbie opinion: if price break 1.3200, it m...      4
25  2021-02-14  Guys, how long do u suppose this place of resi...      4
..         ...                                                ...    ...
118 2021-04-13  Im not selling a penny till we beat the SEC an...      2
117 2021-04-13                                   Here we go again      2
116 2021-04-13                                              HODL🚀      2
115 2021-04-13                                              Bitso      2
337 2022-11-18  Exactly! Even 10% of swift business puts this ...      2

[338 rows x 3 columns]
----------------------------------------------------

In [165]:
# Join text data together retaining date, source and comment
text_data = pd.concat([cleaned_investingcom_xrp_2, cleaned_cryptocompare_xrp_2])[['date', 'source', 'comment']]

# create a new dataframe with the count of comments by date and source
count_data = text_data.groupby(['date', 'source']).size().reset_index(name='count')

# calculate the average number of posts per day - all sources
avg_posts_per_day = count_data.groupby('date')['count'].mean().mean()
print('Average number of posts per day: {:.2f}'.format(avg_posts_per_day))

# calculate the median number of posts per day - all sources
med_posts_per_day = count_data.groupby('date')['count'].median().median()
print('Median number of posts per day: {:.2f}'.format(med_posts_per_day))

Average number of posts per day: 24.84
Median number of posts per day: 8.50


In [158]:
# Qucik EDA to undertsand distrubtion of 
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive

# define a function that plots the count of comments for a given date range
def plot_comments(start_date, end_date):
    start_date = np.datetime64(start_date)
    end_date = np.datetime64(end_date)
    data = count_data[(count_data['date'] >= start_date) & (count_data['date'] <= end_date)]
    fig, ax = plt.subplots(figsize=(12, 6))
    for source in data['source'].unique():
        source_data = data[data['source'] == source]
        ax.plot(source_data['date'], source_data['count'], label=source)
    ax.set_xlabel('Date')
    ax.set_ylabel('Number of comments')
    ax.set_title('Number of comments by source and date')
    ax.legend()
    plt.show()


# create a date slider widget
start_date_widget = widgets.DatePicker(description='Start date:', value=min(count_data['date']))
end_date_widget = widgets.DatePicker(description='End date:', value=max(count_data['date']))

# create an interactive plot using the date slider widget
interactive_plot = interactive(plot_comments, start_date=start_date_widget, end_date=end_date_widget)
output = interactive_plot.children[-1]
output.layout.height = '400px'
interactive_plot


interactive(children=(DatePicker(value=Timestamp('2021-02-02 00:00:00'), description='Start date:'), DatePicke…

### Graph Intepreation & Next steps:
During the period between March and April of 2020-2021, there were significant developments in the legal battle between SEC and Ripple, which is reflected in the online forum discussions. This increase in discussions was particularly noticeable on the Cryptocompare platform, with a substantial spike during that period. For more information on the SEC vs. Ripple lawsuit, refer to this article: https://cointelegraph.com/learn/the-sec-vs-ripple-lawsuit-everything-you-need-to-know.

#### At this stage we have commpleted the following tasks:
- Filtered all data 
- Remove missing observations
- Remove duplicates
- Joined text data together & quick EDA of sample distrubtion

#### The next stages will invlove:
- Text cleanining (unwanted characters, HTML elements, conver to lower case etc)
- Tokenization (Tokenize the text data into indivdual workds or tokens)
- Stopword removal (debating as this can loose meaning of word)
- Lemmatization or stemming (debating)
- Embedding matrix?